In [1]:
# reading in packages
import pandas as pd
import re
import json

In [2]:
# reading in databases
food_details = pd.read_excel('Datasets/Release 1 - Food details file.xlsx')

food_nutrients_excel = pd.ExcelFile('Datasets/Release 1 - Food nutrient database.xlsx')
food_nutrients = pd.read_excel(food_nutrients_excel,'All solids & liquids per 100g')

food_measures = pd.read_excel('Datasets/Release 1 - Measures file.xlsx')
food_recipe = pd.read_excel('Datasets/Release 1 - Recipe file.xlsx')

food_emissions = pd.read_excel('Datasets/emissions.xlsx')
food_categories = pd.read_excel('Datasets/categories.xlsx')

# Exploratory Data Analysis
1534 unique ingredients

503 uniue recipes

268 unique classifications (ie. types?)

# Formatting the Food Databases

In [3]:
# to store all of the food product classifications, now we can drop Classification Name from the food database
food_classification = food_details[['Classification ID', 'Classification Name']].copy()
food_classification.drop_duplicates(inplace = True)

# dropping unecessary columns from food_details
food_details.drop(['Nitrogen Factor', 'Fat Factor', 'Specific Gravity', 'Analysed portion', 'Unanalysed portion', 'Classification Name'], axis = 1, inplace = True)

# dropping unecessary columns from food_nutrients
food_nutrients.drop(['Food Name', 'Classification'], axis = 1, inplace = True)

# need to fix up the merged columns (ie. make the "Unnamed" column match the previous column name) and remove the units row (make a part of column name)
regex = re.compile(r'Unnamed') # regex to find the "Unnamed" columns

old_columns = list(food_nutrients.columns) # list of existing columns
units = food_nutrients[0:1].values[0] # list of the unit row
new_columns = [] # empty list to store the new column values

for each in range(len(old_columns)):
    if re.search(regex, food_nutrients.columns[each]): # ie. find the column that is "unnamed"
        try:
            new_columns.append(old_columns[each-1] + ' (in ' + units[each] + ')') # take the previous column name and add units
        except:
            new_columns.append(old_columns[each-1]) # if you can't add the units, just keep the column
    else: # ie. column is not "unnamed"
        try:
            new_columns.append(old_columns[each] + ' (in ' + units[each] + ')')
        except:
            new_columns.append(old_columns[each])
    
food_nutrients.rename(columns = dict(zip(old_columns, new_columns)), inplace = True) # rename the columns
food_nutrients.drop(0, axis = 0, inplace = True) # drop the unit row
food_nutrients.reset_index(drop = True, inplace = True) # reset the index

# Food name useless (can just get from food_details) and Quantity is 1 everywhere, so drop
food_measures.drop(['Food Name', 'Quantity'], axis = 1, inplace = True)

# Ingredient name useless (we can just get from food_details)
food_recipe.drop(['Ingredient Name'], axis = 1, inplace = True)
food_recipe.rename(columns = {"Public Food Key":"Recipe Public Food Key", "IngredientPublic Food Key":"Public Food Key"}, inplace = True) # rename the columns

# join food_details and food_nutrients
food_details = food_details.merge(food_nutrients, on = 'Public Food Key')


# Exporting the tables out to CSV

In [4]:
#food_categories.to_csv(r'Datasets/tables/food_categories.csv', index = False)
#food_emissions.to_csv('Datasets/tables/food_emissions.csv', index = False)
#food_measures.to_csv('Datasets/tables/food_measures.csv', index = False)
#food_recipe.to_csv('Datasets/tables/food_recipe.csv', index = False)
#food_classification.to_csv('Datasets/tables/food_classification.csv', index = False)
#food_details.to_csv('Datasets/tables/food_details.csv', index = False)

# Setting Product ID and Category ID in food details
For context, I have manually inserted Product ID and Category ID into the "food_classification" data-set, however Product ID and Category ID should be allocated at an individual ingredient level (not at a classification level). The classification level was just done to take care of the initial bulk of allocation, but I need to move Product ID and Category ID into the "food_details" dataset.

In [5]:
# reading in the manual classifications I have created
initial_classification = pd.read_csv("Datasets/food_classification INITIAL.csv")

In [6]:
# need to join food details and classification (on classification ID)
initial_classification['Product ID'] = initial_classification['Product ID'].astype(object) # need to convert to object so there are no floats
food_details = food_details.merge(initial_classification, on = 'Classification ID')

# need to drop unecessary Classification Name column
food_details.drop(['Classification Name'], axis = 1, inplace = True)

food_details.to_csv('Datasets/tables/food_details.csv', index = False)
food_classification.to_csv('Datasets/tables/food_classification.csv', index = False)

In [7]:
#food_categories
#food_classification
#food_details
#food_emissions
#food_measures
#food_recipe

# Customer

In [8]:
# create user JSON
customer = {}

# setting user variables
fname = 'David'
lname = 'Assatrian'
full_name = fname + ' ' + lname
phone = '0410000050'
email = 'dassatrian@gmail.com'

# To add element --> customer['First Name'] = 'David'
# To remove element --> customer.pop('First Name')
customer['First Name'] = fname
customer['Last Name'] = lname
customer['Full Name'] = full_name
customer['Phone'] = phone
customer['Email'] = email
customer['Meals'] = []

json.dumps(customer)

'{"First Name": "David", "Last Name": "Assatrian", "Full Name": "David Assatrian", "Phone": "0410000050", "Email": "dassatrian@gmail.com", "Meals": []}'